In [3]:
# ## dependencies for colab
# !git clone https://__TOKEN_GIT__:@github.com/DanielSc4/RewardLM.git
# %cd RewardLM/
# !pip install -r requirements.txt
# from huggingface_hub import login
# login(token = '')  # https://huggingface.co/settings/tokens

In [8]:
# disable welcome message
import os
os.environ['BITSANDBYTES_NOWELCOME'] = '1'
# os.environ['TOKENIZERS_PARALLELISM'] = 'true'
# os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

# 🥞 Reinforcement Learning with Automatic Feedback (RLAF)

Generation config from [here](https://github.com/LAION-AI/Open-Assistant/blob/main/model/model_training/configs/ppo_config.yaml) seems to work

In [ ]:
# test
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('AlekseyKorshuk/vicuna-7b')

In [18]:
import torch
from rewardlm.core.RL.RLModel import RLModel
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
from transformers import GenerationConfig
from rewardlm.utils import load_config

config = load_config('debug_RL')

rlmanager = RLModel(
    model_id = config['generation']['model_id'],
    reward_model_id = config['reward']['model_id'],
    optimized = True,   # use LoRA
    bs = config['PPO']['bs'],
    mini_bs = config['PPO']['mini_bs'],
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
    generation_config=GenerationConfig(
        max_new_tokens = 512,
        min_new_tokens = 5,
        pad_token_id = 0,       # crashes while using batchsize > 1 only on mps device if not set
        temperature = 1,
        top_p = .7,
        top_k = 0,
        do_sample = True
        # diversity_penalty = .1, # should use num_beam_groups > 1
    )
)

data = get_DIALOCONAN_prepro(delete_last_assistant_response = True)
if config['data']['subset']:
    # select only the first `subset_size` samples
    data = data[:config['data']['subset_size']]
dataset = rlmanager.generate_dataset(text = data)

stats = rlmanager.train_PPO(dataset = dataset)
print('Done')

# assuming debug if subset is active
if not config['data']['subset']:
    # save trainer (model, tokenizer & config) to the hub
    repo_id = 'DanielSc4/' + config['generation']['model_id'].split('/')[1] + '-RL-LoRA-test0'

    rlmanager.push_generator_to_hub(repo_id = repo_id)

Selected device: cpu


/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/peft/tuners/lora.py:240: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 589824 || all params 125030400 || trainable(%): 0.47


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

loader len: 2


100%|██████████| 1/1 [00:00<00:00,  2.67it/s]
1it [00:11, 11.76s/it]

PPO trainer checkpoint saved


100%|██████████| 1/1 [00:00<00:00,  3.87it/s]
2it [00:25, 12.55s/it]


# 👨🏼‍🏫 Model fine-tune

In [2]:
import torch
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
from rewardlm.core.GenerativeModel import GenerativeModel
from rewardlm.data.CustomDatasets import PromptDataset_CLM

from rewardlm.utils import load_config

config = load_config('debug_FT')

generator_manager = GenerativeModel(
    config['generation']['model_id'],
    load_dtype = '8-bit' if torch.cuda.is_available() else 'fp32',
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
)

# download dataset
data = get_DIALOCONAN_prepro()
if config['data']['subset']:
    print('getting subset')
    # select only the first `subset_size` samples
    data = data[:config['data']['subset_size']]
dataset = PromptDataset_CLM(
    tokenizer = generator_manager.tokenizer,
    text = data,
)

generator_manager.fine_tune(
    torch_dataset = dataset, 
    optimized = True, # if torch.cuda.is_available() else False,
    lr = config['generation']['lr'],
    epochs = config['generation']['epochs'],
    initial_bs = config['generation']['initial_bs'],
)


# assuming debug if subset is active
if not config['data']['subset']:
    repo_id = 'DanielSc4/' + config['generation']['model_id'].split('/')[1] + '-FT-LoRA-test0'
    # save model to the hub
    generator_manager.push_to_hub(repo_id = repo_id)

ValueError: AcceleratorState has already been initialized and cannot be changed, restart your runtime completely and pass `cpu=True` to `Accelerator()`.

# ⚖️ ToxicityMeter

In [ ]:
from transformers import GenerationConfig
from rewardlm.ToxicityMeter import ToxicityMeter
from rewardlm.utils.general_utils import device_selector

model_id = 'bigscience/bloomz-3b'
reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target'
batchsize = 16
generation_config = GenerationConfig(
    max_new_tokens = 25,
    num_beams = 5,
    early_stopping = True,
    # crashes while using batchsize > 1 on mps device if not set:
    pad_token_id = 0,
    temperature = 0.8,
    top_p = .8,
    # diversity_penalty = .1, # should use num_beam_groups > 1
)

load_dtype = 'bf16' # can be ['8-bit', 'bf16', 'fp32']
toxicity_meter = ToxicityMeter(
    model_id = model_id,
    load_dtype = load_dtype,
    toxicity_model_id = reward_model_id,
    device = device_selector(),
    generation_config = generation_config,
)

In [ ]:
from rewardlm.data.data_utils import get_real_toxicity_prompts

custom_prompt = ('Statement: "{prompt}".\nResponse: ')
# custom prompts required by the original paper of RedPajama
if model_id == 'togethercomputer/RedPajama-INCITE-Chat-3B-v1':
    custom_prompt = ('<human>: "{prompt}"\n<bot>: ')

toxicity_df = toxicity_meter.measure_toxicity(
    text_prompt = get_real_toxicity_prompts()['text'].to_list(),
    custom_prompt = custom_prompt, 
    generation_config = generation_config,
    batch_size = batchsize,
)

In [ ]:
# save csv in tmp folder
fld = './result analysis/tmp'
toxicity_df.to_csv(
    fld + f'/measured_tox_instruct_{model_id.split("/")[-1]}_{load_dtype}.csv'
)